In [1]:
!pip install tensorflow


In [3]:
import os
import shutil
import random

# Read-only input dir
original_data_dir = '/kaggle/input/trash-data/dataset-resized'
# Writable output dir
output_dir = '/kaggle/working/dataset-split'
split_ratio = 0.8

# Split
categories = os.listdir(original_data_dir)
for category in categories:
    src = os.path.join(original_data_dir, category)
    images = os.listdir(src)
    random.shuffle(images)

    split = int(len(images) * split_ratio)
    train_imgs = images[:split]
    val_imgs = images[split:]

    for subset, subset_imgs in zip(['train', 'val'], [train_imgs, val_imgs]):
        dest = os.path.join(output_dir, subset, category)
        os.makedirs(dest, exist_ok=True)
        for img in subset_imgs:
            shutil.copy(os.path.join(src, img), os.path.join(dest, img))

print("✅ Dataset split recreated successfully.")


✅ Dataset split recreated successfully.


In [4]:
import os

print("✔ Top level:", os.listdir("/kaggle/working"))
print("✔ Train folders:", os.listdir("/kaggle/working/dataset-split/train"))
print("✔ Val folders:", os.listdir("/kaggle/working/dataset-split/val"))


✔ Top level: ['.virtual_documents', 'dataset-split']
✔ Train folders: ['trash', 'paper', 'cardboard', 'metal', 'glass', 'plastic']
✔ Val folders: ['trash', 'paper', 'cardboard', 'metal', 'glass', 'plastic']


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
import numpy as np
import os

# -------------------- SETUP --------------------
TRAIN_DIR = "/kaggle/working/dataset-split/train"
VAL_DIR = "/kaggle/working/dataset-split/val"
cnn_model_path = "/kaggle/working/waste_cnn_classifier.h5"
mobilenet_model_path = "/kaggle/working/waste_classifier_mobilenetv2.h5"
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 10
NUM_CLASSES = 6
class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# -------------------- DATA LOADERS --------------------
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)
val_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(TRAIN_DIR, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical')
val_ds = val_gen.flow_from_directory(VAL_DIR, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

# -------------------- CNN MODEL --------------------
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])
cnn_model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[EarlyStopping(patience=3, restore_best_weights=True)])
cnn_model.save(cnn_model_path)

# -------------------- MOBILENETV2 MODEL --------------------
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)
mobilenet_model = Model(inputs=base_model.input, outputs=output)

mobilenet_model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
mobilenet_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[EarlyStopping(patience=3, restore_best_weights=True)])
mobilenet_model.save(mobilenet_model_path)

# -------------------- ENSEMBLE PREDICTION --------------------
def get_dustbin_category(waste_type):
    if waste_type in ['plastic', 'glass', 'metal', 'cardboard', 'paper']:
        return 'Recyclables ♻️'
    elif waste_type == 'trash':
        return 'Landfill 🚮'
    else:
        return 'Unknown'

def ensemble_predict(image_path):
    img = Image.open(image_path).convert("RGB").resize((IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    cnn_preds = cnn_model.predict(img_array)
    mobilenet_preds = mobilenet_model.predict(img_array)
    final_preds = (cnn_preds + mobilenet_preds) / 2.0

    class_idx = np.argmax(final_preds)
    confidence = round(np.max(final_preds) * 100, 2)
    waste_type = class_names[class_idx]
    bin_type = get_dustbin_category(waste_type)

    return waste_type, bin_type, confidence



Found 2019 images belonging to 6 classes.
Found 508 images belonging to 6 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 32s 210ms/step - accuracy: 0.2378 - loss: 1.7450 - val_accuracy: 0.3858 - val_loss: 1.4692
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 23s 177ms/step - accuracy: 0.3860 - loss: 1.5487 - val_accuracy: 0.3957 - val_loss: 1.4176
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 22s 177ms/step - accuracy: 0.4063 - loss: 1.4185 - val_accuracy: 0.4291 - val_loss: 1.4063
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 23s 179ms/step - accuracy: 0.4678 - loss: 1.3274 - val_accuracy: 0.4882 - val_loss: 1.2831
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 22s 176ms/step - accuracy: 0.4776 - loss: 1.2920 - val_accuracy: 0.4902 - val_loss: 1.2290
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 23s 179ms/step - accuracy: 0.4929 - loss: 1.2443 - val_accuracy: 0.5472 - val_loss: 1.2081
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 22s 175ms/step - accuracy: 0.5371 - loss: 1.1900 - val_accuracy: 0.5650 - val_loss: 1.1364
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 22s 174ms/step - accuracy: 0.5595 - loss: 1

In [12]:

# -------------------- EXAMPLE --------------------
example_image = "/kaggle/input/trash-data/dataset-resized/cardboard/cardboard10.jpg"  # Change this path
if os.path.exists(example_image):
    waste, bin_type, conf = ensemble_predict(example_image)
    print(f"🔍 Waste: {waste} | 🗑️ Bin: {bin_type} | ✅ Confidence: {conf}%")
else:
    print("⚠️ Example image not found. Update the path.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
🔍 Waste: cardboard | 🗑️ Bin: Recyclables ♻️ | ✅ Confidence: 63.53%
